In [1]:
import requests
from time import sleep
from datetime import date
import pandas as pd

In [2]:
newsrooms = pd.read_csv('../data/raw/newsrooms.csv')
newsrooms.columns = ['site', 'monthly_visits', 'country', 'country_of_pub']
newsrooms.dropna(inplace=True)
newsrooms = newsrooms.drop_duplicates(subset = 'site')
newsrooms = newsrooms.sort_values(by=['country_of_pub', 'monthly_visits'], ascending=False).reset_index().drop('index', axis=1)

In [3]:
UK = newsrooms[newsrooms['country_of_pub']=='UK'].reset_index().drop('index', axis=1)
USA = newsrooms[newsrooms['country_of_pub']=='USA'].reset_index().drop('index', axis=1)
India = newsrooms[newsrooms['country_of_pub']=='India'].reset_index().drop('index', axis=1)
SA = newsrooms[newsrooms['country_of_pub']=='South Africa'].reset_index().drop('index', axis=1)

In [4]:
all_sites = list(SA.site.unique().flatten()) + list(USA.site.unique().flatten()) + list(UK.site.unique().flatten()) + list(India.site.unique().flatten())

In [5]:
len(all_sites)

203

In [5]:
len(UK) + len(USA) + len(India) + len(SA)

203

In [6]:
year = 2020

def get_two_month_ranges(year):
    month_ranges = [[[1, 1, year], [31, 1, year]], 
                    [[1, 2, year], [28, 2, year]],
                    [[1, 3, year], [31, 3, year]],
                    [[1, 4, year], [30, 4, year]],
                    [[1, 5, year], [31, 5, year]],
                    [[1, 6, year], [30, 6, year]],
                    [[1, 7, year], [31, 7, year]],
                    [[1, 8, year], [31, 8, year]],
                    [[1, 9, year], [30, 9, year]],
                    [[1, 10, year], [31, 10, year]],
                    [[1, 11, year], [30, 11, year]],
                    [[1, 12, year], [31, 12, year]]]

    two_month_ranges = []
    for i in range(len(month_ranges)-1):
        if i%2==0:
            two_month_ranges.append((month_ranges[i][0], month_ranges[i+1][1]))

    return two_month_ranges

In [7]:
month_ranges = []
for i in range(2008,2020):
    month_ranges = month_ranges + get_two_month_ranges(i)

In [8]:
import tqdm

In [10]:
# month_ranges

In [12]:
%%time

from tqdm import tqdm
headlines_dict = {}

sites = all_sites
# sites = list(USA['site'].unique().flatten())[1:]

url = "https://google-news.p.rapidapi.com/v1/source_search"
# url = "https://google-news1.p.rapidapi.com/search"

request_count = 0
for i in tqdm(month_ranges):
    from_date = f'{i[0][2]}-{i[0][1]}-{i[0][0]}'
    to_date = f'{i[1][2]}-{i[1][1]}-{i[1][0]}'
    year = i[0][2]
    bimester = i[1][1]/2

    for site in sites:

        print("Scraping" ,site, "starting from", from_date, "to", to_date)

        urls = []
        headlines = []
        times = []
        scrape_dates = []
        websites = []

        # baseline news

        #querystring = { "lang":"en","from":from_date,"to":to_date,"source":site}

        # women's news

        querystring = {"q": "women OR woman OR girl OR female OR lady OR ladies OR she OR her OR herself OR aunt OR grandmother OR mother OR sister", 
                       "lang":"en","from":from_date,"to":to_date,"source":site}

        headers = {
            # Paid API
            'x-rapidapi-key': "bd5010b7eemsh00ab5684fa6a7efp15b791jsnd00e2d098829",
            # Free API
            #'x-rapidapi-key': "3e81132ebdmshafd93b150db164ep13c18fjsn834550268d59",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }

        response = requests.request("GET", url, headers=headers, params=querystring).json()

        request_count += 1

        for j in range(len(response['articles'])):
            headlines.append(response['articles'][j]['title'])
            times.append(response['articles'][j]['published'])
            urls.append(response['articles'][j]['link'])
            websites.append(response['articles'][j]['source']['href'])

        headlines_dict = {"url": urls, "headline": headlines, "time": times, "scrape_date": date.today().strftime("%d/%m/%Y"), "site": site}

        headlines_df = pd.DataFrame.from_dict(headlines_dict)
        number_of_articles = len(headlines_df)
        # save results to csv
        if number_of_articles!=0:
            
            headlines_df.to_csv(f'../data/raw_temporal/{number_of_articles}_{site}_{bimester}_{year}_articles.csv')
        # print final statement
            print("Saved data from " + str(number_of_articles) + " articles of "  + str(site) + " to .csv!")
        else:
            print("No data from " + str(site) + " during this timeframe!")
            
        print(request_count, "requests fired")


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]

Scraping News24.com starting from 2008-1-1 to 2008-2-28
No data from News24.com during this timeframe!
1 requests fired
Scraping Gumtree.co.za starting from 2008-1-1 to 2008-2-28
No data from Gumtree.co.za during this timeframe!
2 requests fired
Scraping Cars.co.za starting from 2008-1-1 to 2008-2-28
No data from Cars.co.za during this timeframe!
3 requests fired
Scraping BusinessTech.co.za starting from 2008-1-1 to 2008-2-28
No data from BusinessTech.co.za during this timeframe!
4 requests fired
Scraping Iol.co.za starting from 2008-1-1 to 2008-2-28
No data from Iol.co.za during this timeframe!
5 requests fired
Scraping Maroelamedia.co.za starting from 2008-1-1 to 2008-2-28
No data from Maroelamedia.co.za during this timeframe!
6 requests fired
Scraping Timeslive.co.za starting from 2008-1-1 to 2008-2-28
No data from Timeslive.co.za during this timeframe!
7 requests fired
Scraping Ewn.co.za starting from 2008-1-1 to 2008-2-28
No data from Ewn.co.za during this timeframe!
8 requests fi

  1%|█                                                                               | 1/72 [05:52<6:56:59, 352.39s/it]

No data from economictimes.indiatimes.com during this timeframe!
203 requests fired
Scraping News24.com starting from 2008-3-1 to 2008-4-30
No data from News24.com during this timeframe!
204 requests fired
Scraping Gumtree.co.za starting from 2008-3-1 to 2008-4-30
No data from Gumtree.co.za during this timeframe!
205 requests fired
Scraping Cars.co.za starting from 2008-3-1 to 2008-4-30
No data from Cars.co.za during this timeframe!
206 requests fired
Scraping BusinessTech.co.za starting from 2008-3-1 to 2008-4-30
No data from BusinessTech.co.za during this timeframe!
207 requests fired
Scraping Iol.co.za starting from 2008-3-1 to 2008-4-30
No data from Iol.co.za during this timeframe!
208 requests fired
Scraping Maroelamedia.co.za starting from 2008-3-1 to 2008-4-30
No data from Maroelamedia.co.za during this timeframe!
209 requests fired
Scraping Timeslive.co.za starting from 2008-3-1 to 2008-4-30
No data from Timeslive.co.za during this timeframe!
210 requests fired
Scraping Ewn.co.

  3%|██▏                                                                             | 2/72 [11:38<6:48:45, 350.36s/it]

...

Saved data from 47 articles of economictimes.indiatimes.com to .csv!
14210 requests fired
Scraping News24.com starting from 2019-9-1 to 2019-10-31
Saved data from 100 articles of News24.com to .csv!
14211 requests fired
Scraping Gumtree.co.za starting from 2019-9-1 to 2019-10-31
No data from Gumtree.co.za during this timeframe!
14212 requests fired
Scraping Cars.co.za starting from 2019-9-1 to 2019-10-31
No data from Cars.co.za during this timeframe!
14213 requests fired
Scraping BusinessTech.co.za starting from 2019-9-1 to 2019-10-31
Saved data from 2 articles of BusinessTech.co.za to .csv!
14214 requests fired
Scraping Iol.co.za starting from 2019-9-1 to 2019-10-31
Saved data from 100 articles of Iol.co.za to .csv!
14215 requests fired
Scraping Maroelamedia.co.za starting from 2019-9-1 to 2019-10-31
No data from Maroelamedia.co.za during this timeframe!
14216 requests fired
Scraping Timeslive.co.za starting from 2019-9-1 to 2019-10-31
Saved data from 100 articles of Timeslive.co.za t

 99%|█████████████████████████████████████████████████████████████████████████████▉ | 71/72 [7:00:25<06:38, 398.78s/it]

Saved data from 40 articles of economictimes.indiatimes.com to .csv!
14413 requests fired
Scraping News24.com starting from 2019-11-1 to 2019-12-31
Saved data from 100 articles of News24.com to .csv!
14414 requests fired
Scraping Gumtree.co.za starting from 2019-11-1 to 2019-12-31
No data from Gumtree.co.za during this timeframe!
14415 requests fired
Scraping Cars.co.za starting from 2019-11-1 to 2019-12-31
No data from Cars.co.za during this timeframe!
14416 requests fired
Scraping BusinessTech.co.za starting from 2019-11-1 to 2019-12-31
No data from BusinessTech.co.za during this timeframe!
14417 requests fired
Scraping Iol.co.za starting from 2019-11-1 to 2019-12-31
Saved data from 100 articles of Iol.co.za to .csv!
14418 requests fired
Scraping Maroelamedia.co.za starting from 2019-11-1 to 2019-12-31
No data from Maroelamedia.co.za during this timeframe!
14419 requests fired
Scraping Timeslive.co.za starting from 2019-11-1 to 2019-12-31
Saved data from 100 articles of Timeslive.co.

100%|███████████████████████████████████████████████████████████████████████████████| 72/72 [7:07:06<00:00, 355.93s/it]

Saved data from 44 articles of economictimes.indiatimes.com to .csv!
14616 requests fired
Wall time: 7h 7min 6s


In [11]:
all_sites

['News24.com',
 'Gumtree.co.za',
 'Cars.co.za',
 'BusinessTech.co.za',
 'Iol.co.za',
 'Maroelamedia.co.za',
 'Timeslive.co.za',
 'Ewn.co.za',
 'Thesouthafrican.com',
 'Businessinsider.co.za',
 'Citizen.co.za',
 'Netwerk24.com',
 'Dailymaverick.co.za',
 'Vodacom.co.za',
 'Sowetanlive.co.za',
 'mg.co.za',
 'MyBroadband.co.za',
 'Moneyweb.co.za',
 'Businesslive.co.za',
 'Enca.com',
 'Capetownetc.com',
 'iol.co.za',
 'heraldlive.co.za',
 'dispatchlive.co.za',
 'dailysun.co.za',
 'cbn.co.za',
 'lowvelder.co.za',
 'zululandobserver.co.za',
 'ziwaphi.com',
 'drudgereport.com',
 'espn.go.com',
 'usatoday.com',
 'nbcnews.com',
 'popsugar.com',
 'abcnews.go.com',
 'ksl.com',
 'nydailynews.com',
 'politico.com',
 'sfgate.com',
 'npr.org',
 'cbsnews.com',
 'rawstory.com',
 'people.com',
 'seekingalpha.com',
 'tmz.com',
 'latimes.com',
 'bleacherreport.com',
 'pitchfork.com',
 'cnbc.com',
 'gizmodo.com',
 'wsj.com',
 'complex.com',
 'nypost.com',
 'refinery29.com',
 'slate.com',
 'fivethirtyeight.c

In [154]:
# response['articles']

In [156]:
to_date

'2005-2-28'

In [18]:
url = "https://google-news.p.rapidapi.com/v1/source_search"

querystring = {"q": "women OR woman OR girl OR female OR lady OR ladies OR she OR her OR herself OR wife OR mom OR mother OR sister", 
                       "lang":"en","from":"2005-1-1","to":"2005-2-28","source":"economictimes.indiatimes.com"}

# "women OR woman OR girl OR female OR lady OR ladies OR she OR her OR herself OR aunt OR grandmother OR mother OR sister OR daughter OR wife OR mom OR mum OR girlfriend OR mrs OR niece"        
headers = {
    'x-rapidapi-key': "bd5010b7eemsh00ab5684fa6a7efp15b791jsnd00e2d098829",
    'x-rapidapi-host': "google-news.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring).json()

# print(response.text)

In [19]:
response

{'feed': {'title': '"allinurl:economictimes.indiatimes.com women OR woman OR girl OR female OR lady OR ladies OR she OR her OR herself OR wife OR mom OR mother OR sister after:2005-01-01 before:2005-02-28" - Google News',
  'updated': 'Fri, 14 May 2021 21:19:29 GMT',
  'link': 'https://news.google.com/search?q=allinurl:economictimes.indiatimes.com+women+OR+woman+OR+girl+OR+female+OR+lady+OR+ladies+OR+she+OR+her+OR+herself+OR+wife+OR+mom+OR+mother+OR+sister+after:2005-01-01+before:2005-02-28&ceid=US:en&hl=en-US&gl=US',
  'language': 'en-US',
  'subtitle': 'Google News',
  'rights': '2021 Google Inc.'},
 'articles': []}